### Ignore the following cell

In the following cell I create the fake output that I will return later in the notebook

In [107]:
import pandas as pd

import os
from binascii import hexlify

def get_key():
    key = hexlify(os.urandom(32)).decode()
    return key

class Grid():
    ""
class Wallet():
    ""
gr = Grid()
gr.wallet = Wallet()

#ignore this...it's just to support the mock API
columns=["network", "domain", "pubkey", "prikey"]
data = [["OpenGrid", "PatrickCason", get_key(), get_key()],
       ["OpenGrid", "AndrewTrask", get_key(), get_key()],
       ["OpenGrid", "TudorCebere", get_key(), get_key()],
       ["OpenGrid", "JasonMancuso", get_key(), get_key()],
       ["OpenGrid", "BobbyWagner", get_key(), get_key()],
       ["AMA", "UCSF", get_key(), get_key()],
       ["AMA", "Vanderbilt", get_key(), get_key()],
       ["AMA", "MDAnderson", get_key(), get_key()],
       ["AMA", "BostonGeneral", get_key(), get_key()],
       ["AMA", "HCA", get_key(), get_key()],
       ["CDC", "Atlanta", get_key(), get_key()],
       ["CDC", "New York", get_key(), get_key()],
       ]
domain_keys = pd.DataFrame(columns=columns, data=data)
gr.wallet.domain_keys = domain_keys

#ignore this...it's just to support the mock API
columns=["id", "name", "datasets", "models", "domains", "online", "registered"]
data = [[235252, "OpenGrid", 235262, 2352, 2532, 2352, 23],
       [634252, "AMA", 2352, 236622, 53, 52, 23],
       [745742, "CDC", 35, 0, 5, 5, 5]]
networks = pd.DataFrame(columns=columns, data=data)
networks = networks.set_index("name")    
gr.networks = networks

def register_network(network):
    columns=["id", "name", "datasets", "models", "domains", "online", "registered"]
    data = [[2352, "NHS", 86585, 6585, 5, 5, 0]]
    network = pd.DataFrame(columns=columns, data=data)
    network = network.set_index("name")
    
    gr.networks = pd.concat([gr.networks, network])
    
    return network

gr.register_network = register_network

# Step 1: Imports

To start, from a client perspective, we want to maximize for convenience and minimize the number of dependencies one needs to install to work with PyGrid. Thus, in an ideal world, users only have to install one python package in order to work with all of pygrid. I like the current design in syft 0.2.x where we have grid clients in a grid package inside of Syft. The thing we definitely want to avoid here is the need for users of PyGrid to have to install all of the dependencies needed to _run grid nodes_ (flask, databases, etc.) just to be able to interact with the grid. Putting grid inside of syft solves this as well.

In [108]:
# import syft as sy
# from syft import grid as gr

# Step 2: Default Networks

By default, it would be really great if we could support a combination of two lists of networks:

- networks which all users of PySyft have by default (OpenGrid)
- a history of all networks previously accessed (stored in some local config file)

We should be able to view these available networks by just calling `gr.networks` which should pretty-print information about them. Below we show one way to do pretty-print using just a Pandas table as shown below.

In [109]:
gr.networks

,id,datasets,models,domains,online,registered
name,,,,,,
OpenGrid,235252,235262,2352,2532,2352,23
AMA,634252,2352,236622,53,52,23
CDC,745742,35,0,5,5,5


This table displays several useful pieces of information:

- Id: the unique id of the network
- Name: the name of the network
- Datasets: the number of datasets currently hosted on the network
- Models: the number of models currently hosted on the network (public and private)
- Domains: the number of domains registered to this network (i.e., the number of individual hospitals)
- Online: the number of domains which are currently online
- Registered: the number of domains for which this user already has an account 

# Step 3: Local Wallet

Somewhere in the local filesystem, we need to save the set of all keys/logins which this user has for various domains around the world. We should be able to see them here. Note this list is what creates the "Registered" number for each network.

In [111]:
gr.wallet.domain_keys

,network,domain,pubkey,prikey
0,OpenGrid,PatrickCason,3050a44dececd60ed9b9a83e0686c790b549a5efb0942e...,e7b63e129989e4d7798a6ff04460b2543a750dcbbbfc49...
1,OpenGrid,AndrewTrask,7bb7426516861839411f876a92e9be1576713f36d37cf1...,e676b6289e0ae297cdc1756fc1333ef11153a79b9a2cd0...
2,OpenGrid,TudorCebere,0416e84fc2dae81273a89d551d2110e76954882a52a83a...,9b3cb304860a919835e226944de900086ff80398b470e0...
3,OpenGrid,JasonMancuso,5c0059f27a98887ce60df720a0f71e60e82b92c03db767...,49e00243430f564134e3fd08072ae445d41aa2238bcf6e...
4,OpenGrid,BobbyWagner,7bc596089903897b7d79696e8973c2abe67a7201386031...,da980f9f8d11e5e7f35717b787f89a8875e907a1aee1a0...
5,AMA,UCSF,422ea39fd3e8a9fea41d23ac32066ba9f1bbbd1d1b1e6f...,fa491634660ff0ac8602099ee1a0786390bdd398895050...
6,AMA,Vanderbilt,7034f9eb725665a5c634ae2d618bc9f28105e576d042cb...,df55c650777d6350773b25ac52da542637f42aad6a5622...
7,AMA,MDAnderson,addf55e6aa13e5c3248e5afe0b4850a14478d7215fa4b3...,6ea615c281ac88b6b096fa0d732dce0e0eb7dc62b0440a...
8,AMA,BostonGeneral,13468dc24deac30ee93194bc9b8ec0cf51366ed3962be5...,ba1e8f387da60d223b3f587ace13aa4e2a5d974a788fae...
9,AMA,HCA,49e7e8f4c316adb34708b227085e9572a157c9dbe73c65...,6120fbf4735a1c741adb80767c6dd595cd88f1c3f8b5e9...


# Step 4: Adding Another Network

We should be able to add another network by simply dropping in the url to the network node (much like adding another PyPI/Npm repository or something)

In [112]:
gr.register_network('ws://nhs.co.uk/pygrid') # it's a network

,id,datasets,models,domains,online,registered
name,,,,,,
NHS,2352,86585,6585,5,5,0


In [113]:
gr.networks

,id,datasets,models,domains,online,registered
name,,,,,,
OpenGrid,235252,235262,2352,2532,2352,23
AMA,634252,2352,236622,53,52,23
CDC,745742,35,0,5,5,5
NHS,2352,86585,6585,5,5,0


In [ ]:


# diabetesSearch = network.search('diabetes') # search dataset name, description, and tags for 'diabetes'
diabetesSearch = network.search({ tag: 'diabetes' }) # specifically search for datasets with a tag of 'diabetes'

print(diabetesSearch)

"""
[
  {
    id: 1,
    name: 'Diabetes is terrible',
    description: '',
    node: 'ws://ucsf.com/pygrid',
    tags: ['diabetes', 'california', 'ucsf'],
    tensors: [
      {
        id: '1a',
        name: 'data',
        schema: []
      },
      {
        id: '1b',
        name: 'target',
        schema: []
      }
    ]
  },
  ...
]
"""

network.disconnect()

client = grid.connect(diabetesSearch[0].node) # 'ws://ucsf.com/pygrid'

user = client.signup('me@patrickcason.com', 'password')
# user = client.login('me@patrickcason.com', 'password')  # or, if you're already signed up

computeTypes = client.getComputeTypes()

"""
[
  {
    id: 1,
    name: 'EC2 P3',
    provider: 'AWS',
    cpu: {
      type: 'Intel Xeon 3.4GHz',
      cores: 32
    },
    gpu: {
      type: 'Tesla V100',
      min: 0,
      max: 8
    },
    ram: {
      value: 64,
      ordinal: 'gb'
    }
  },
  ...
]
"""

# env = user.createEnvironment() # creates the basic "default" environment for exploring

env = user.createEnvironment(computeTypes[0].id, {
    ram: Grid.RAM(32, 'gb'),
    gpu: 3
})

# Do stuff with "env"

# user.getEnvironments();